In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import mass_estimation as me
from tqdm import tqdm

## Read in the data

In [ ]:
#Define some file paths
git_folder_loc = '/home/stba7609/'
model_tracks_path = 'NII_nebular_phase/prerequisites/model_tracks/'
all_tracks_path = 'NII_nebular_phase/prerequisites/observed_tracks/'
Ib_tracks_path = 'SECRETO/WISEREP_Ib/Ib_tracks/'
Ic_tracks_path = 'SECRETO/WISEREP_Ic/Ic_tracks/'
IIb_tracks_path = 'SECRETO/WISEREP_IIb/IIb_tracks/'

In [ ]:
#Create arrays with file names for the different types
Ib_names = np.array(['SN1985F', 'SN1996aq', 'SN2004ao', 'SN2009jf',
                    'SN2015ah', 'SN2004dk', 'SN2012au', 'SNMASTEROTJ1204'
                     , 'SN2008D', 'SN2007Y', 'SN2007C', 'SN2006ld', 'SN2004gq',
                    'SN2019odp'])
Ic_names = np.array(['SN1987M', 'SN1997dq', 'SN1998bw', 'SN2003gf', 'SN2004aw', 'SN2007gr', 'SN2007I',
                    'SN2011bm', 'SN2013ge', 'SN2015fn', 'SN2019yz', 'SNPTF12gzk'])
IIb_names = np.array(['SN1993J', 'SN1996cb', 'SN2001ig', 'SN2003bg', 'SN2008ax',
                      'SN2008aq', 'SN2008bo', 'SN2009K', 'SN2011dh', 'SN2011ei', 'SN2011hs',
                      'SN2013df', 'SN2020acat','SN2022crv', 'SNASASSN14az', 'SNPTF12os', 'iPTF13bvn'])

all_names = np.concatenate((Ib_names, Ic_names, IIb_names))

In [ ]:
#Read in the model tracks
string_masses = np.array(['3p3', '4p0', '5p0', '6p0', '8p0']) #Write as 'xpx'
model_masses = np.array([3.3, 4.0, 5.0, 6.0, 8.0])
model_tracks = []
for i in range(len(string_masses)):
    track = np.loadtxt(git_folder_loc + model_tracks_path + 'NII_track_he' + string_masses[i] + '.csv',
              delimiter = ',')
    model_tracks.append(track)

## Get the masses

In [ ]:
#The cell doing all the work
all_m_pred_low, all_m_pred_mid, all_m_pred_high = np.zeros(len(all_names)), np.zeros(len(all_names)), np.zeros(len(all_names))
for i in tqdm(range(len(all_names))):
    
    sn_name = all_names[i]
    
    #Read in the track
    obs_track = np.loadtxt(git_folder_loc + all_tracks_path + 'NII_track_' + sn_name + '.csv',
              delimiter = ',') 
    
    #Mask out values before a given epoch
    cut_epoch = 200 #Days post explosion
    
    
    #Get the mass
    print('Getting mass for ', sn_name)
    
    if len(obs_track.shape) > 1:
        
        epoch_mask = np.where(obs_track[:, 0] > cut_epoch)[0]
        obs_track = obs_track[epoch_mask]
        
        if obs_track == [] or len(obs_track) == 0: #No epochs before cut off, so no estimate can be made
            mass_68_left, best_mass, mass_68_right = -1, -1 , -1
        else:
            best_mass, mass_68_left, mass_68_right = me.mass_determinator_mcmc(obs_track[:, 0], obs_track[:, 1], obs_track[:, 2], model_tracks=model_tracks, plot = True)
    else:
        if obs_track[0] < cut_epoch: #No epochs before cut off, so no estimate can be made
            mass_68_left, best_mass, mass_68_right = -1, -1 , -1
        else:
            best_mass, mass_68_left, mass_68_right = me.mass_determinator_mcmc(np.array([obs_track[0]]), np.array([obs_track[1]]), np.array([obs_track[2]]), model_tracks=model_tracks, plot = True)
    
    all_m_pred_low[i] = mass_68_left
    all_m_pred_mid[i] = best_mass
    all_m_pred_high[i] = mass_68_right

## Make a nice plot

In [ ]:
#Lets read in the predicted masses from the literature

metadata_Ib = np.loadtxt(git_folder_loc + 'SECRETO/WISEREP_Ib/metadata.csv',
              delimiter = ',', dtype = str)
metadata_Ic = np.loadtxt(git_folder_loc + 'SECRETO/WISEREP_Ic/metadata.csv',
              delimiter = ',', dtype = str)
metadata_IIb = np.loadtxt(git_folder_loc + 'SECRETO/WISEREP_IIb/metadata.csv',
              delimiter = ',', dtype = str)

metadata_all = np.concatenate((metadata_Ib, metadata_Ic, metadata_IIb))

In [ ]:
#First, we create a plot for Mf vs Mf
fig, ax = plt.subplots(1)

colours = ['orange', 'red', 'green', 'blue']
sn_types = ['Ib', 'Ic', 'IIb', 'eIIb']
literature_errs_list, prediction_errs_list, plot_inds = [], [], []
m_pred_mid_f_list, m_lit_mid_f_list, accepted_names = [], [], []
no_lit, lim_list = False, []

for i in range(len(metadata_all)):
    literature_errs, predictions_errs = 0, 0
    lolims, uplims = None, None #These are for plotting errorbars later
    
    lit_name = metadata_all[i, 0]
    
    try:
        prediction_index = np.where(lit_name == all_names )[0][0]
    except:
        continue
        
    accepted_names.append(lit_name)
    
    #Define the predicted Mhe values for this object
    m_pred_low, m_pred_mid, m_pred_high = all_m_pred_low[prediction_index], all_m_pred_mid[prediction_index], all_m_pred_high[prediction_index] 
    m_pred_low_f = me.Mhe_i_to_Mhe_f(m_pred_low)
    m_pred_mid_f = me.Mhe_i_to_Mhe_f(m_pred_mid)
    m_pred_high_f = me.Mhe_i_to_Mhe_f(m_pred_high)
    
    #Get the literature values for this object
    if metadata_all[i, -2] != '--' and metadata_all[i, -1] != '--': #If Mhe,i is available, translate Mhe,i to Mf
        m_he_low, m_he_high = float(metadata_all[i, -2]), float(metadata_all[i, -1])
        m_lit_low_f, m_lit_high_f = me.Mhe_i_to_Mhe_f(m_he_low), me.Mhe_i_to_Mhe_f(m_he_high)
        m_lit_mid_f = (m_lit_low_f + m_lit_high_f)/2
        
    elif lit_name not in IIb_names and metadata_all[i, 7] != '--' and metadata_all[i, 8] != '--': #If Mej is available, use that
        m_ej_low, m_ej_high = float(metadata_all[i, 7]), float(metadata_all[i, 8])
        m_lit_low_f, m_lit_high_f = me.Mej_to_Mhe_f(m_ej_low), me.Mej_to_Mhe_f(m_ej_high)
        m_lit_mid_f = (m_lit_low_f + m_lit_high_f)/2
    else:
        no_lit = True
        m_lit_low_f, m_lit_mid_f, m_lit_high_f = -5, -5, -5
     
            
            
    #Check for plotting if we have lower or upper limits
    offset = 0
    this_ms = 10
    if m_pred_mid >= 8: #As our grid stops here, our predictions can only give lower limits
        m_pred_low_f, m_pred_mid_f, m_pred_high_f = m_pred_low_f, m_pred_low_f, m_pred_low_f
        
        this_marker = '^'

        lim_list.append(2)
            
    elif m_pred_mid <= 3.3: #As our grid stops here, our predictions can only give upper limits
        m_pred_low_f, m_pred_mid_f, m_pred_high_f = m_pred_high_f, m_pred_high_f, m_pred_high_f #No errs, as this is an upper limit
        
        this_marker = 'v'

        lim_list.append(1)
    else:
        this_marker = '.'
        this_ms = 15
        
        lim_list.append(0)
        
    prediction_errs = np.array([[m_pred_mid_f-m_pred_low_f,
                                     m_pred_high_f-m_pred_mid_f]]).T
    
    if no_lit == True:
        pass
    else:
        literature_errs = np.array([[m_lit_mid_f-m_lit_low_f, 
                                          m_lit_high_f-m_lit_mid_f]]).T
        
        
    #This is put in by hand, as it saves a lot of code that only would serve 2 SNe:
    if lit_name == 'SN2011bm':
        this_marker = '>'
    elif lit_name == 'SN2015fn':
        this_marker = 'h'
        
            
    #Give the right colour
    if lit_name in Ib_names:
        plot_ind = 0
    elif lit_name in Ic_names:
        plot_ind = 1
    elif lit_name in IIb_names:
        plot_ind = 2
    
    if no_lit == True:
        literature_errs_list.append([-5, -5])
        m_lit_mid_f_list.append(-5)
    else:
        literature_errs_list.append(literature_errs)
        m_lit_mid_f_list.append(m_lit_mid_f)
        
        ax.errorbar(x = m_lit_mid_f,
                    y = m_pred_mid_f+offset,
                    yerr = prediction_errs,
                    xerr = literature_errs,
                    fmt = '.', ms = this_ms, lw = 1, marker = this_marker, capsize = 5,
                    color = colours[plot_ind], alpha = 0.8)
        

    #Make sure that things with no prediction also get a -1 assigned:
    if all_m_pred_low[prediction_index] == -1:
        prediction_errs = np.array([[0, 
                                     0]]).T
        m_pred_mid_f = -1
        
        
    prediction_errs_list.append(prediction_errs)
    plot_inds.append(plot_ind)
    m_pred_mid_f_list.append(m_pred_mid_f)
    
    no_lit = False
    
    
    
    
    
#And now the plot specifics

#Getting the labels
ax.scatter(x = -1, y = -1, label = 'Ib', color = colours[0])
ax.scatter(x = -1, y = -1, label = 'Ic', color = colours[1])
ax.scatter(x = -1, y = -1, label = 'IIb', color = colours[2])

#Shade the areas not outside of our working range
ax.fill_between(x = [2, 8.5], y1 = 5.64, y2 = 8.5, color = 'gray', alpha = 0.5)
ax.fill_between(x = [2, 8.5], y1 = 2, y2 = 2.67, color = 'gray', alpha = 0.5)
    
ax.set_xlim(2, 8.5)
ax.set_ylim(2, 8.5)
ax.plot(np.linspace(ax.get_xlim()[0], ax.get_xlim()[1], 100), np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], 100), linestyle = '--', c = 'black')
ax.set_xlabel('Literature $M_{preSN}$ [M$_{\odot}$]')
ax.set_ylabel('Predicted $M_{preSN}$ [M$_{\odot}$]')
ax.grid()


ax.legend(fontsize = 12, frameon = True)
fig.set_size_inches(4, 4)
plt.savefig('Figure_10.pdf', dpi = 300, bbox_inches = 'tight')
plt.show()           

## Store all the masses

In [ ]:
final_product = np.zeros((len(all_names), 9), dtype = object)

#Safety check:
if len(all_names) != len(metadata_all):
    print('WARNING: You may be missing some SN data')

#First add the data which does have literature predictions
for i in range(len(accepted_names)):
    
    final_product[i, 0] = accepted_names[i]
    final_product[i, 1] = sn_types[plot_inds[i]]
    final_product[i, 2] = '{:.1f}'.format(m_pred_mid_f_list[i])
    final_product[i, 3] = '{:.1f}'.format(prediction_errs_list[i][0][0])
    final_product[i, 4] = '{:.1f}'.format(prediction_errs_list[i][1][0])
    
    if m_lit_mid_f_list[i] == -5: #This means there are no literature vals:
        final_product[i, 5] = '--'
        final_product[i, 6] = '--'
        final_product[i, 7] = '--'
        
    else: #There exist literature valls!
        final_product[i, 5] = '{:.1f}'.format(m_lit_mid_f_list[i])
        final_product[i, 6] = '{:.1f}'.format(literature_errs_list[i][0][0])
        final_product[i, 7] = '{:.1f}'.format(literature_errs_list[i][1][0])
        
    final_product[i, 8] = lim_list[i]
    
print(final_product)
fmt = '%s,%s,%s,%s,%s,%s,%s,%s,%s'
    
np.savetxt('mass_predictions.csv', final_product, fmt = fmt, delimiter = ',', header = '## name,type,m_pred_mid_f,m_pred_low_f,m_pred_high_f,m_lit_mid_f,m_lit_low_f,m_lit_high_f')
                                          